In [6]:
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, GroupKFold

In [7]:
df_train = pd.read_csv('train_with_features.csv')

## TODO: fill with 0 or median depending on the column

In [8]:
df_train = df_train.fillna(0)

In [9]:
kfold = GroupKFold()


In [10]:
df_train.stock_id = df_train.stock_id.astype('category')

In [11]:
cont_nn,cat_nn = cont_cat_split(df_train, max_card=9000, dep_var='target')

In [12]:
cat_nn.remove('time_id'), cat_nn.remove('row_id')

(None, None)

In [20]:
def train_fold(trn_idx, val_idx, fname):
    

    procs_nn = [Categorify, Normalize]
    to_nn = TabularPandas(df_train, procs_nn, cat_nn, cont_nn,
                          splits=[list(trn_idx), list(val_idx)], y_names='target')

    dls = to_nn.dataloaders(1024)
    learn = tabular_learner(dls, y_range=(0,.1), layers=[400,100,500],config={'lin_first':False, 'embed_p':.2, 'ps':[.1,.1,0]},
                        n_out=1, loss_func=rmspe_loss, metrics=AccumMetric(rmspe),
                        #cbs=[EarlyStoppingCallback(monitor='rmspe', comp=np.less, patience=20),
                        #    SaveModelCallback(monitor='rmspe', comp=np.less, fname=fname)]
                           )
    learn.fit_one_cycle(50, 3e-3)
    learn.save(fname)
    return learn.recorder.metrics[0].value

In [14]:
def rmspe(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [15]:
def rmspe_loss(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [ ]:
res =[]
for trn_idx, val_idx in kfold.split(df_train, groups = df_train.time_id):
    res.append(train_fold(trn_idx, val_idx, f'kfold_{len(res)}'))
    print('fold', len(res), res[-1])
np.mean(res),res

epoch,train_loss,valid_loss,rmspe,time
0,0.314066,0.250724,0.253712,00:03
1,0.249555,0.238166,0.241371,00:03
2,0.238963,0.235773,0.237726,00:03
3,0.243068,0.229764,0.231957,00:03
4,0.243234,0.248261,0.250325,00:03
5,0.247904,0.242199,0.244040,00:03
6,0.251702,0.235722,0.238401,00:02
7,0.245307,0.241246,0.243723,00:03
8,0.254524,0.247101,0.252060,00:03
9,0.241680,0.231857,0.234378,00:03


fold 1 TensorBase(0.2228)


epoch,train_loss,valid_loss,rmspe,time
0,0.321014,0.249242,0.251558,00:03
1,0.249422,0.235189,0.237871,00:03
2,0.243731,0.237350,0.238970,00:03
3,0.245589,0.233209,0.235347,00:03
4,0.242847,0.236275,0.238005,00:03
5,0.243861,0.239750,0.242927,00:03
6,0.240692,0.237297,0.239061,00:03
7,0.240730,0.236452,0.241047,00:03
8,0.244704,0.235419,0.237472,00:03
9,0.237830,0.236313,0.239743,00:03


fold 2 TensorBase(0.2273)


epoch,train_loss,valid_loss,rmspe,time
0,0.310790,0.242586,0.244734,00:02
1,0.249687,0.230138,0.232745,00:02
2,0.248322,0.241026,0.243025,00:02
3,0.244352,0.238946,0.240749,00:03
4,0.247225,0.233235,0.237628,00:03
5,0.244686,0.253164,0.257267,00:03
6,0.245725,0.237585,0.239543,00:03
7,0.246069,0.227107,0.229357,00:03
8,0.244241,0.234860,0.236747,00:03
9,0.243863,0.221880,0.223730,00:03


fold 3 TensorBase(0.2145)


epoch,train_loss,valid_loss,rmspe,time
0,0.311232,0.246701,0.248815,00:03
1,0.252212,0.230919,0.233427,00:03
2,0.244453,0.228313,0.232016,00:03
3,0.245931,0.234231,0.239986,00:03
4,0.247985,0.234971,0.236800,00:03
5,0.239622,0.223434,0.225203,00:03
6,0.245778,0.235367,0.238352,00:03
7,0.250973,0.241236,0.244651,00:03
8,0.247582,0.224021,0.226876,00:03
9,0.251469,0.229054,0.232121,00:03


fold 4 TensorBase(0.2133)


epoch,train_loss,valid_loss,rmspe,time
0,0.307614,0.235318,0.237845,00:03
1,0.251872,0.226749,0.229237,00:02
2,0.242704,0.220978,0.223244,00:02
3,0.250135,0.223868,0.226809,00:02
4,0.241990,0.225774,0.227548,00:02
5,0.246242,0.241529,0.246980,00:02
6,0.259862,0.238530,0.245379,00:03
7,0.251211,0.229440,0.231042,00:03
8,0.251266,0.252875,0.257901,00:02
9,0.245918,0.234157,0.236670,00:03
